# Stage 3: Deployment Demo

## Deployment Overview

In this notebook, we demonstrate how to deploy our final churn prediction model to new, unseen customer data.  
This replicates a real-world scenario where the business can identify customers at risk of churn and take proactive retention measures.

In [33]:
# Load necessary libraries
import pandas as pd
import numpy as np
import joblib

## Load Final Model, Scaler, and Columns

We load our saved Logistic Regression model, scaler for numerical features, and the list of feature columns to ensure that incoming data is pre-processed consistently.

In [34]:
# Paths (update to your local Desktop path if needed)
model_path = '/Users/Cathaml/Desktop/final_lr_model.pkl'
scaler_path = '/Users/Cathaml/Desktop/final_scaler.pkl'
columns_path = '/Users/Cathaml/Desktop/feature_columns.pkl'

# Load model, scaler, and columns
lr = joblib.load(model_path)
scaler = joblib.load(scaler_path)
feature_columns = joblib.load(columns_path)

## Create Example New Customer Data

We create a small sample dataset representing new customers.  
In a real production scenario, this data would come from your CRM or operational database.

In [35]:
# Create new customer dataframe (example raw input)
new_customers_raw = pd.DataFrame({
    'gender': ['Female', 'Male', 'Male'],
    'SeniorCitizen': [0, 1, 0],
    'Partner': ['Yes', 'No', 'No'],
    'Dependents': ['No', 'No', 'Yes'],
    'tenure': [1, 12, 24],
    'PhoneService': ['Yes', 'Yes', 'No'],
    'MultipleLines': ['No', 'Yes', 'No phone service'],
    'InternetService': ['DSL', 'Fibre optic', 'DSL'],
    'OnlineSecurity': ['No', 'Yes', 'No'],
    'OnlineBackup': ['Yes', 'No', 'Yes'],
    'DeviceProtection': ['No', 'Yes', 'No'],
    'TechSupport': ['No', 'No', 'Yes'],
    'StreamingTV': ['No', 'Yes', 'No'],
    'StreamingMovies': ['No', 'Yes', 'No'],
    'Contract': ['Month-to-month', 'Two year', 'One year'],
    'PaperlessBilling': ['Yes', 'Yes', 'No'],
    'PaymentMethod': ['Electronic check', 'Mailed check', 'Bank transfer (automatic)'],
    'MonthlyCharges': [29.85, 70.70, 56.95],
    'TotalCharges': [29.85, 151.65, 1889.50]
})

## Pre-process New Customer Data

We encode binary columns, perform one-hot encoding, align columns to match the model, and scale numerical features using the saved scaler.

In [36]:
# Map binary columns
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    new_customers_raw[col] = new_customers_raw[col].map({'Yes': 1, 'No': 0})

# One-hot encode
new_customers_encoded = pd.get_dummies(new_customers_raw, drop_first=True)

# Align columns exactly
new_customers_aligned = new_customers_encoded.reindex(columns=feature_columns, fill_value=0)

# Scale numerical columns
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
new_customers_aligned[num_cols] = scaler.transform(new_customers_aligned[num_cols])

## Make Predictions

We use the loaded Logistic Regression model to generate churn predictions and churn probabilities for each new customer.

In [37]:
# Make predictions
predictions = lr.predict(new_customers_aligned)
pred_probs = lr.predict_proba(new_customers_aligned)[:, 1]

# Add results to original dataframe
new_customers_raw['PredictedChurn'] = predictions
new_customers_raw['ChurnProbability'] = np.round(pred_probs, 2)

# Show results
new_customers_raw[['PredictedChurn', 'ChurnProbability']]

,PredictedChurn,ChurnProbability
0,1,0.54
1,0,0.17
2,0,0.07


## 💬 Analysis: Deployment Results

In this final deployment demo, we successfully use the pre-trained Logistic Regression model to make churn predictions for new customers.

- **PredictedChurn** indicates whether the customer is expected to churn (`1`) or stay (`0`).
- **ChurnProbability** provides an interpretable probability score between 0 and 1.
- By combining both outputs, teams can prioritise retention efforts and design targeted interventions for high-risk customers.

---

This notebook demonstrates a complete, production-ready end-to-end pipeline, from pre-processing to scoring, ready to be integrated into a live system or business workflow.